In [11]:
from tpot import TPOTRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data = pd.read_csv("new_train.csv")
train = data.drop(['trip_duration', 'log_trip_duration' ],axis=1)
train.head()


,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_month,pickup_dayofweek,pickup_hour,pickup_second,maximum temerature,minimum temperature,average temperature,precipitation,snow fall,snow depth,distance,pickup_cluster,dropoff_cluster
0,2,1,-73.982155,40.767937,-73.964630,40.765602,3,0,17,6369878.0,51,40,45.5,0.29,0.0,0.0,1.502172,16,24
1,1,1,-73.975090,40.758766,-73.953201,40.765068,3,0,14,6357922.0,51,40,45.5,0.29,0.0,0.0,1.976279,19,6
2,1,1,-73.994484,40.745087,-73.998993,40.722710,3,0,15,6361461.0,51,40,45.5,0.29,0.0,0.0,2.513961,20,23
3,2,3,-73.944359,40.714489,-73.910530,40.709492,3,0,4,6323059.0,51,40,45.5,0.29,0.0,0.0,2.911908,17,13
4,2,1,-73.952881,40.766468,-73.978630,40.761921,3,0,14,6361059.0,51,40,45.5,0.29,0.0,0.0,2.231986,6,19


In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(train[:25000], data['log_trip_duration'][:25000], test_size=0.2, random_state=43)

In [4]:
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=5)
tpot.fit(X_train, y_train)
tpot.export('tpot_exported_pipeline.py')

GP Progress:   3%|▋                    | 4/120 [01:48<1:07:56, 35.14s/pipeline]C:\Users\Aditya\Anaconda2\envs\tensorflow_win\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
GP Progress:   6%|█▎                     | 7/120 [02:08<36:20, 19.30s/pipeline]C:\Users\Aditya\Anaconda2\envs\tensorflow_win\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. I

In [5]:
print(tpot.score(X_valid, y_valid))


C:\Users\Aditya\Anaconda2\envs\tensorflow_win\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


0.139338846749


In [7]:
pred = tpot.predict(X_valid)

In [12]:
print(rmsle(np.exp(y_valid.values), np.exp(pred)))
print(np.sqrt(mean_squared_error(np.exp(y_valid),np.exp(pred))))
print(r2_score(np.exp(y_valid), np.exp(pred)))

0.370265573071
275.02300232
0.819632769023


In [9]:
def rmsle(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5